# Formatters

In [1]:
import re
import operator

import pandas as pd

## MMRotate benchmarks (website)

In [2]:
tables = pd.read_html('https://mmrotate.readthedocs.io/en/latest/model_zoo.html')
assert len(tables) == 1
df = tables[0]

In [3]:
df[
    (df['Backbone'] == 'ResNet50 (1024,1024,200)')
    & (df['Angle'] == 'le90')
    & ~df['Configs'].str.contains('_hbb_|_fp16_',)
    & df['Inf Time (fps)'].notna()
]

,Backbone,mAP,Angle,lr schd,Mem (GB),Inf Time (fps),Aug,Batch Size,Configs,Download
4,"ResNet50 (1024,1024,200)",68.42,le90,1x,3.38,16.9,-,2,rotated_retinanet_obb_r50_fpn_1x_dota_le90,model | log
17,"ResNet50 (1024,1024,200)",70.64,le90,1x,3.12,18.2,-,2,rotated_atss_obb_r50_fpn_1x_dota_le90,model | log
27,"ResNet50 (1024,1024,200)",73.23,le90,1x,8.45,16.4,-,2,gliding_vertex_r50_fpn_1x_dota_le90,model | log
28,"ResNet50 (1024,1024,200)",73.40,le90,1x,8.46,16.5,-,2,rotated_faster_rcnn_r50_fpn_1x_dota_le90,model | log
33,"ResNet50 (1024,1024,200)",75.69,le90,1x,8.46,16.2,-,2,oriented_rcnn_r50_fpn_1x_dota_le90,model | log
36,"ResNet50 (1024,1024,200)",76.08,le90,1x,8.67,14.4,-,2,roi_trans_r50_fpn_1x_dota_le90,model | log


## MMRotate Metric Logs

In [4]:
rretinanet_log = '''\
+--------------------+-------+--------+--------+-------+
| class              | gts   | dets   | recall | ap    |
+--------------------+-------+--------+--------+-------+
| plane              | 4449  | 12021  | 0.911  | 0.876 |
| baseball-diamond   | 358   | 5249   | 0.902  | 0.757 |
| bridge             | 783   | 26870  | 0.566  | 0.341 |
| ground-track-field | 212   | 9351   | 0.892  | 0.596 |
| small-vehicle      | 10579 | 114057 | 0.845  | 0.655 |
| large-vehicle      | 8819  | 71059  | 0.825  | 0.664 |
| ship               | 18537 | 48742  | 0.865  | 0.777 |
| tennis-court       | 1512  | 9414   | 0.947  | 0.905 |
| basketball-court   | 266   | 4469   | 0.793  | 0.614 |
| storage-tank       | 4740  | 23401  | 0.684  | 0.605 |
| soccer-ball-field  | 251   | 5593   | 0.685  | 0.485 |
| roundabout         | 275   | 6147   | 0.782  | 0.631 |
| harbor             | 4167  | 23615  | 0.719  | 0.585 |
| swimming-pool      | 732   | 9325   | 0.697  | 0.524 |
| helicopter         | 122   | 9987   | 0.656  | 0.372 |
+--------------------+-------+--------+--------+-------+
| mAP                |       |        |        | 0.626 |
+--------------------+-------+--------+--------+-------+'''

orcnn_log = '''\
+--------------------+-------+-------+--------+-------+
| class              | gts   | dets  | recall | ap    |
+--------------------+-------+-------+--------+-------+
| plane              | 4449  | 5150  | 0.929  | 0.893 |
| baseball-diamond   | 358   | 591   | 0.866  | 0.754 |
| bridge             | 783   | 1945  | 0.681  | 0.511 |
| ground-track-field | 212   | 709   | 0.901  | 0.776 |
| small-vehicle      | 10579 | 27731 | 0.843  | 0.692 |
| large-vehicle      | 8819  | 16161 | 0.924  | 0.848 |
| ship               | 18537 | 22256 | 0.939  | 0.892 |
| tennis-court       | 1512  | 1931  | 0.943  | 0.908 |
| basketball-court   | 266   | 594   | 0.872  | 0.742 |
| storage-tank       | 4740  | 4627  | 0.688  | 0.626 |
| soccer-ball-field  | 251   | 940   | 0.833  | 0.648 |
| roundabout         | 275   | 600   | 0.785  | 0.683 |
| harbor             | 4167  | 6600  | 0.820  | 0.740 |
| swimming-pool      | 732   | 1173  | 0.745  | 0.579 |
| helicopter         | 122   | 353   | 0.705  | 0.558 |
+--------------------+-------+-------+--------+-------+
| mAP                |       |       |        | 0.723 |
+--------------------+-------+-------+--------+-------+'''

In [5]:
def extract(s: str):
    log_pattern = re.compile(r'''
        .*?(?P<class>[-\w]+)
        .*?(?P<gts>\d+)
        .*?(?P<dets>\d+)
        .*?(?P<recall>-?\d+.\d+)
        .*?(?P<ap>-?\d+.\d+)
        ''', re.VERBOSE)
    dtypes = str, int, int, float, float
    return [[d(x) for d, x in zip(dtypes, m)] for m in log_pattern.findall(s)]


cols_all = ['Class', 'Ground Truths', 'Detections', 'Recall', 'Average Precision']
rretinanet = pd.DataFrame.from_records(extract(rretinanet_log), columns=cols_all).set_index('Class')
orcnn = pd.DataFrame.from_records(extract(orcnn_log), columns=cols_all).set_index('Class')

keys = ('Rotated RetinaNet', 'Oriented RCNN')
cols_filter = ['Detections', 'Recall', 'Average Precision']
df = pd.concat(
    [rretinanet[cols_filter], orcnn[cols_filter]],
    keys=keys,
    axis=1
)
df = df.swaplevel(axis=1).reindex(cols_filter, axis=1, level=0).reindex(keys, axis=1, level=1)
df.columns = df.columns.to_flat_index().str.join(' ')
df = pd.concat([rretinanet[['Ground Truths']], df], axis=1)
df.index = df.index.str.replace('-', ' ').str.title()

mean = df.mean(axis=0)
mean.name = 'Mean'
df = df.append(mean)

df


/tmp/ipykernel_31255/2318001797.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(mean)


,Ground Truths,Detections Rotated RetinaNet,Detections Oriented RCNN,Recall Rotated RetinaNet,Recall Oriented RCNN,Average Precision Rotated RetinaNet,Average Precision Oriented RCNN
Class,,,,,,,
Plane,4449.000000,12021.000000,5150.000000,0.9110,0.9290,0.8760,0.893000
Baseball Diamond,358.000000,5249.000000,591.000000,0.9020,0.8660,0.7570,0.754000
Bridge,783.000000,26870.000000,1945.000000,0.5660,0.6810,0.3410,0.511000
Ground Track Field,212.000000,9351.000000,709.000000,0.8920,0.9010,0.5960,0.776000
Small Vehicle,10579.000000,114057.000000,27731.000000,0.8450,0.8430,0.6550,0.692000
Large Vehicle,8819.000000,71059.000000,16161.000000,0.8250,0.9240,0.6640,0.848000
Ship,18537.000000,48742.000000,22256.000000,0.8650,0.9390,0.7770,0.892000
Tennis Court,1512.000000,9414.000000,1931.000000,0.9470,0.9430,0.9050,0.908000
Basketball Court,266.000000,4469.000000,594.000000,0.7930,0.8720,0.6140,0.742000


In [6]:
df.reset_index().to_excel('/mnt/c/users/konpa/desktop/metrics.xlsx', index=False)